Processing Data

In [8]:
import pandas as pd
import numpy as np
import sqlite3

connection = sqlite3.connect("focus.db")
df = pd.read_sql("SELECT * FROM focus_logs",connection)
#The first thing is to add a tracking ended entry. So...
df

,id,date,time,focused,program
0,1,09/05/2024,16:42:38,focus.py - TimeLens - Visual Studio Code,Code.exe
1,2,09/05/2024,16:42:45,processing.ipynb - TimeLens - Visual Studio Code,Code.exe
2,3,09/05/2024,16:42:46,main.py - TimeLens - Visual Studio Code,Code.exe
3,4,09/05/2024,16:42:49,focus.py - TimeLens - Visual Studio Code,Code.exe


In [2]:
#First DF

# At the beginning of all of this, in the raw csv, add a entry signifying the Tracking Ended. This is so we don't have
# crazy gaps between ending and starting the next session. Add an ignorance for it.
from datetime import datetime

new_df = pd.DataFrame()
new_df['Program'] = df['Program'].unique()
#Taking all programs from the csv
new_df #printing
# Now by entry, I want to get a total time/add up datetimes. Join "Date" and "Time" then just sum up?
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
#Makes a column w full datetime object

# Sort by 'DateTime' to ensure correct time difference calculation
df = df.sort_values(by=['DateTime'])

# Current datetime - next datetime
df['TimeSpent'] = df['DateTime'].shift(-1) - df['DateTime']
# For the last row, set a default time (e.g., 5 seconds), as there's no next row
df.iloc[-1, df.columns.get_loc('TimeSpent')] = pd.Timedelta(seconds=5)

# Group by 'Program' and sum the total time spent for each program
total_time_per_program = df.groupby('Program')['TimeSpent'].sum()
print(total_time_per_program)


Program
Code.exe     0 days 00:01:52
chrome.exe   1 days 00:49:11
Name: TimeSpent, dtype: timedelta64[ns]
